## 仅供交叉验证 Encoder-Decoder with LSTM cell -按趋势分类

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [3]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [4]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [46]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按趋势划分

In [2]:
def split_data_by_trend(data, targets):
    up_data = []
    down_data = []
    up_target = []
    down_target = []
    
    for i in range(len(data)):
        a, b = np.polyfit(range(len(data[i])), data[i, :, -2].reshape(-1), 1)
        if a > 0:
            up_data.append(data[i])
            up_target.append(targets[i])
        else:
            down_data.append(data[i])
            down_target.append(targets[i])
    return np.array(up_data), np.array(up_target), np.array(down_data), np.array(down_target)

### 构建模型

In [3]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

def build_encoder_decoder_model(lstm_units, dense_units, lr=1e-4):
    model = keras.models.Sequential()
    model.add(LSTM(lstm_units, activation='tanh', input_shape=(11, 10), return_sequences=True))
    model.add(LSTM(lstm_units, activation='tanh'))
    model.add(RepeatVector(5))
    model.add(LSTM(lstm_units, activation='tanh', return_sequences=True))
    model.add(LSTM(lstm_units, activation='tanh', return_sequences=True))
    model.add(TimeDistributed(Dense(dense_units, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    
    optimizer=keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }
    
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2 = split_data_by_trend(X_train, y_train)
        train_xs = [X_train1, X_train2]
        train_ys = [y_train1, y_train2]
        
        X_test1, y_test1, X_test2, y_test2 = split_data_by_trend(X_test, y_test)
        test_xs = [X_test1, X_test2]
        test_ys = [y_test1, y_test2]
        i_s = [1, 2]
        
        # 训练
        for i in range(len(i_s)):
            model = build_encoder_decoder_model(256, 256, 1e-4)
            history = model.fit(train_xs[i], train_ys[i], epochs=100, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('rnn_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 2054 samples, validate on 524 samples
Epoch 1/100
2054/2054 [==============================] - 5s 3ms/sample - loss: 1.0014 - val_loss: 0.9604
Epoch 2/100
2054/2054 [==============================] - 1s 477us/sample - loss: 0.9678 - val_loss: 0.9429
Epoch 3/100
2054/2054 [==============================] - 1s 722us/sample - loss: 0.9513 - val_loss: 0.9218
Epoch 4/100
2054/2054 [==============================] - 3s 1ms/sample - loss: 0.9313 - val_loss: 0.9126
Epoch 5/100
2054/2054 [==============================] - 3s 1ms/sample - loss: 0.9152 - val_loss: 0.8964
Epoch 6/100
2054/2054 [==============================] - 3s 1ms/sample - loss: 0.9092 - val_loss: 0.8946
Epoch 7/100
2054/2054 [==============================] - 3s 1ms/sample - loss: 0.9062 - val_loss: 0.8962
Epoch 8/100
2054/2054 [==============================] - 3s 1ms/sample - loss: 0.9028 - val_loss: 0.8886
Epoch 9/100
205

2065/2065 [==============================] - 3s 2ms/sample - loss: 0.8892 - val_loss: 0.9159
Epoch 18/100
2065/2065 [==============================] - 3s 2ms/sample - loss: 0.8886 - val_loss: 0.9150
Epoch 19/100
2065/2065 [==============================] - 3s 2ms/sample - loss: 0.8886 - val_loss: 0.9101
Epoch 20/100
2065/2065 [==============================] - 4s 2ms/sample - loss: 0.8877 - val_loss: 0.9092
Epoch 21/100
2065/2065 [==============================] - 4s 2ms/sample - loss: 0.8878 - val_loss: 0.9098
Epoch 22/100
2065/2065 [==============================] - 4s 2ms/sample - loss: 0.8881 - val_loss: 0.9120
Epoch 23/100
2065/2065 [==============================] - 3s 1ms/sample - loss: 0.8880 - val_loss: 0.9155
Epoch 24/100
2065/2065 [==============================] - 4s 2ms/sample - loss: 0.8874 - val_loss: 0.9102
Epoch 25/100
2065/2065 [==============================] - 4s 2ms/sample - loss: 0.8872 - val_loss: 0.9085
Epoch 26/100
2065/2065 [==============================] - 4

2066/2066 [==============================] - 4s 2ms/sample - loss: 0.8930 - val_loss: 0.8882
Epoch 24/100
2066/2066 [==============================] - 3s 2ms/sample - loss: 0.8915 - val_loss: 0.8990
Epoch 25/100
2066/2066 [==============================] - 3s 1ms/sample - loss: 0.8918 - val_loss: 0.8933
Epoch 26/100
2066/2066 [==============================] - 4s 2ms/sample - loss: 0.8920 - val_loss: 0.8977
Epoch 27/100
2066/2066 [==============================] - 2s 937us/sample - loss: 0.8918 - val_loss: 0.8963
Epoch 28/100
2066/2066 [==============================] - 1s 722us/sample - loss: 0.8918 - val_loss: 0.8945
Epoch 29/100
2066/2066 [==============================] - 4s 2ms/sample - loss: 0.8913 - val_loss: 0.8947
Epoch 30/100
2066/2066 [==============================] - 4s 2ms/sample - loss: 0.8904 - val_loss: 0.8932
Epoch 31/100
2066/2066 [==============================] - 4s 2ms/sample - loss: 0.8906 - val_loss: 0.8905
Epoch 32/100
2066/2066 [==============================]

2062/2062 [==============================] - 4s 2ms/sample - loss: 1.0146 - val_loss: 0.9303
Epoch 2/100
2062/2062 [==============================] - 2s 982us/sample - loss: 0.9712 - val_loss: 0.9222
Epoch 3/100
2062/2062 [==============================] - 3s 1ms/sample - loss: 0.9507 - val_loss: 0.9095
Epoch 4/100
2062/2062 [==============================] - 3s 2ms/sample - loss: 0.9315 - val_loss: 0.8968
Epoch 5/100
2062/2062 [==============================] - 2s 873us/sample - loss: 0.9189 - val_loss: 0.8862
Epoch 6/100
2062/2062 [==============================] - 3s 1ms/sample - loss: 0.9137 - val_loss: 0.8802
Epoch 7/100
2062/2062 [==============================] - 3s 1ms/sample - loss: 0.9093 - val_loss: 0.8791
Epoch 8/100
2062/2062 [==============================] - 4s 2ms/sample - loss: 0.9072 - val_loss: 0.8769
Epoch 9/100
2062/2062 [==============================] - 2s 1ms/sample - loss: 0.9055 - val_loss: 0.8729
Epoch 10/100
2062/2062 [==============================] - 4s 2m

54/54 [==============================] - 0s 525us/sample - loss: 0.9876 - val_loss: 0.5961
Epoch 37/100
54/54 [==============================] - 0s 642us/sample - loss: 0.9804 - val_loss: 0.5695
Epoch 38/100
54/54 [==============================] - 0s 633us/sample - loss: 0.9731 - val_loss: 0.5682
Epoch 39/100
54/54 [==============================] - 0s 626us/sample - loss: 0.9737 - val_loss: 0.5672
Epoch 40/100
54/54 [==============================] - 0s 642us/sample - loss: 0.9735 - val_loss: 0.5661
Epoch 41/100
54/54 [==============================] - 0s 530us/sample - loss: 0.9718 - val_loss: 0.5669
Epoch 42/100
54/54 [==============================] - 0s 531us/sample - loss: 0.9712 - val_loss: 0.5670
Epoch 43/100
54/54 [==============================] - 0s 518us/sample - loss: 0.9705 - val_loss: 0.5671
Epoch 44/100
54/54 [==============================] - 0s 518us/sample - loss: 0.9697 - val_loss: 0.5662
Epoch 45/100
54/54 [==============================] - 0s 537us/sample - loss:

...

Epoch 14/100
3542/3542 [==============================] - 6s 2ms/sample - loss: 0.6206 - val_loss: 0.6282
Epoch 15/100
3542/3542 [==============================] - 3s 971us/sample - loss: 0.6195 - val_loss: 0.6304
Epoch 16/100
3542/3542 [==============================] - 5s 1ms/sample - loss: 0.6185 - val_loss: 0.6270
Epoch 17/100
3542/3542 [==============================] - 6s 2ms/sample - loss: 0.6189 - val_loss: 0.6264
Epoch 18/100
3542/3542 [==============================] - 6s 2ms/sample - loss: 0.6188 - val_loss: 0.6273
Epoch 19/100
3542/3542 [==============================] - 6s 2ms/sample - loss: 0.6183 - val_loss: 0.6283
Epoch 20/100
3542/3542 [==============================] - 5s 2ms/sample - loss: 0.6180 - val_loss: 0.6258
Epoch 21/100
3542/3542 [==============================] - 4s 1ms/sample - loss: 0.6170 - val_loss: 0.6269
Epoch 22/100
3542/3542 [==============================] - 5s 1ms/sample - loss: 0.6170 - val_loss: 0.6269
Epoch 23/100
3542/3542 [====================

3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6204 - val_loss: 0.6298
Epoch 13/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6201 - val_loss: 0.6285
Epoch 14/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6196 - val_loss: 0.6294
Epoch 15/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6178 - val_loss: 0.6305
Epoch 16/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6174 - val_loss: 0.6288
Epoch 17/100
3535/3535 [==============================] - 5s 1ms/sample - loss: 0.6176 - val_loss: 0.6274
Epoch 18/100
3535/3535 [==============================] - 4s 1ms/sample - loss: 0.6172 - val_loss: 0.6294
Epoch 19/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6167 - val_loss: 0.6332
Epoch 20/100
3535/3535 [==============================] - 6s 2ms/sample - loss: 0.6167 - val_loss: 0.6284
Epoch 21/100
3535/3535 [==============================] - 6

3515/3515 [==============================] - 6s 2ms/sample - loss: 0.6370 - val_loss: 0.6155
Epoch 4/100
3515/3515 [==============================] - 3s 738us/sample - loss: 0.6334 - val_loss: 0.6112
Epoch 5/100
3515/3515 [==============================] - 5s 1ms/sample - loss: 0.6318 - val_loss: 0.6091
Epoch 6/100
3515/3515 [==============================] - 5s 2ms/sample - loss: 0.6308 - val_loss: 0.6087
Epoch 7/100
3515/3515 [==============================] - 3s 936us/sample - loss: 0.6302 - val_loss: 0.6072
Epoch 8/100
3515/3515 [==============================] - 6s 2ms/sample - loss: 0.6287 - val_loss: 0.6087
Epoch 9/100
3515/3515 [==============================] - 5s 1ms/sample - loss: 0.6278 - val_loss: 0.6089
Epoch 10/100
3515/3515 [==============================] - 4s 1ms/sample - loss: 0.6271 - val_loss: 0.6095
Epoch 11/100
3515/3515 [==============================] - 6s 2ms/sample - loss: 0.6262 - val_loss: 0.6055
Epoch 12/100
3515/3515 [==============================] - 5s 

599/599 [==============================] - 1s 2ms/sample - loss: 0.4046 - val_loss: 0.3851
Epoch 32/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4030 - val_loss: 0.3863
Epoch 33/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4056 - val_loss: 0.3847
Epoch 34/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4014 - val_loss: 0.3969
Epoch 35/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4037 - val_loss: 0.3859
Epoch 36/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4013 - val_loss: 0.3894
Epoch 37/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.4014 - val_loss: 0.3869
Epoch 38/100
599/599 [==============================] - 1s 1ms/sample - loss: 0.4008 - val_loss: 0.3850
Epoch 39/100
599/599 [==============================] - 1s 1ms/sample - loss: 0.4011 - val_loss: 0.3834
Epoch 40/100
599/599 [==============================] - 1s 2ms/sample - loss:

Epoch 6/100
578/578 [==============================] - 0s 483us/sample - loss: 0.4307 - val_loss: 0.4041
Epoch 7/100
578/578 [==============================] - 0s 484us/sample - loss: 0.4259 - val_loss: 0.4020
Epoch 8/100
578/578 [==============================] - 0s 541us/sample - loss: 0.4245 - val_loss: 0.3972
Epoch 9/100
578/578 [==============================] - 0s 691us/sample - loss: 0.4248 - val_loss: 0.3953
Epoch 10/100
578/578 [==============================] - 0s 702us/sample - loss: 0.4197 - val_loss: 0.3965
Epoch 11/100
578/578 [==============================] - 1s 1ms/sample - loss: 0.4190 - val_loss: 0.3924
Epoch 12/100
578/578 [==============================] - 1s 1ms/sample - loss: 0.4177 - val_loss: 0.3917
Epoch 13/100
578/578 [==============================] - 1s 1ms/sample - loss: 0.4211 - val_loss: 0.3902
Epoch 14/100
578/578 [==============================] - 1s 1ms/sample - loss: 0.4153 - val_loss: 0.3892
Epoch 15/100
578/578 [==============================] - 1s

In [ ]:
metrics

In [49]:
overall_metrics

{'mae': [0.7593994116060747,
  0.7671028942138409,
  0.7561637531690426,
  0.7418184505992778,
  0.754409714862091],
 'rmse': [1.2462465377385148,
  1.2449577006458816,
  1.2305761200091225,
  1.2152881768708477,
  1.3016089605016066],
 'ndcg': [0.12383840192943663,
  0.8889930008127779,
  0.5801052823760275,
  0.7835049919448109,
  0.07865601010951555]}